In [5]:
import torch
import os
from neuMF import NeuMF
from dataset_updated import simple_load_data_rate, get_model_data
import random
import torch
import torch.nn as nn
import torch.multiprocessing as mp
mp.set_start_method('spawn', force=True)
# import data as data
# import model_evaluation as evaluation
import torch.optim as optim
import torch._dynamo
import numpy as np
import heapq
from torch.optim.lr_scheduler import ReduceLROnPlateau
from evaluation import model_evaluation, model_evaluation_metric
import pandas as pd
from collections import defaultdict
torch._dynamo.config.suppress_errors = True

model = torch.load('best_model_ncf_layer.pth', weights_only=False)


base_dir = os.getcwd()
name_rating_dir = "ml-1m/ratings.dat"
rating_data_file = os.path.join(base_dir, name_rating_dir)

#train_dict, val_dict, test_dict, non_interacted_movies, movie_num, user_num = load_data_rate(file_name)

train_dict, valid_dict, test_dict, movie_num, user_num, removed_users_info= simple_load_data_rate(rating_data_file, negative_sample_no_train=10,negative_sample_no_valid= 100, threshold=3)

train_user_input, train_movie_input, train_labels = get_model_data(train_dict)
valid_user_input, valid_movie_input, valid_labels = get_model_data(valid_dict)
test_user_input, test_movie_input, test_labels = get_model_data(test_dict)

test_dict = defaultdict(list)
for user_id, movie_id, label in zip(test_user_input, test_movie_input, test_labels):
    test_dict[user_id].append((movie_id, label))
test_dict = dict(test_dict)

with torch.no_grad():
    test_recall, test_ndcg = model_evaluation_metric(model, test_dict, device, K=10)
    print(f"\n=== Test Recall@10: {test_recall:.4f}, Test NDCG@10: {test_ndcg:.4f} ===\n")
    
    results[name] = {
       
        "test_recall": test_recall,
        "test_ndcg": test_ndcg
    }

NameError: name 'device' is not defined